<a href="https://colab.research.google.com/github/marco-siino/ThingSpeak_ParsersGenerator/blob/main/Gemma_7B_Instruct_ThingSpeak_ParsersGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Thu Jun 20 01:27:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               74W / 461W|  10423MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the Python code to perform the task.

In [3]:
prompt ="""
Write the Python code to convert the following JSON FILE using the following JSON SCHEMA:

JSON FILE:

{
    "channel": {
        "id": 1293177,
        "name": "San Diego - Estaci\u00f3n Meteorol\u00f3gica",
        "description": "San Diego, Cerro Largo, Uruguay\r\nEstaci\u00f3n Meteorol\u00f3gica Solar\r\n(Temp, Hum, Presion, Lluvia, Viento).\r\nESP8266, UNO R3, BME 680\r\nUpdate Interval - 15 seg\r\nhttps://clima.santiago.ovh/",
        "latitude": "-31.9939484",
        "longitude": "-53.9575388",
        "field1": "Temperatura C\u00b0",
        "field2": "Humedad %",
        "field3": "Pres. Atmosf\u00e9rica (hPa)",
        "field4": "Gas",
        "field5": "Viento",
        "field6": "Precipitaci\u00f3n (mm)",
        "field7": "Direcci\u00f3n del Viento",
        "field8": "UV",
        "created_at": "2021-01-30T16:32:32Z",
        "updated_at": "2024-06-18T14:05:34Z",
        "elevation": "136",
        "last_entry_id": 4502987
    },
    "feeds": [
        {
            "created_at": "2024-06-18T13:33:39Z",
            "entry_id": 4502888,
            "field1": "17.73",
            "field2": "93.32",
            "field3": "996.29",
            "field4": "103.33",
            "field5": "0.00",
            "field6": "0",
            "field7": "0",
            "field8": "0.74"
        }
    ]
}

JSON SCHEMA:

{
    "type": "object",
    "properties": {
      "sensorId": {
        "type": "string"
      },
      "timestamp": {
        "type": "string",
        "format": "date-time"
      },
      "temperature": {
        "type": "number"
      },
      "unit": {
        "type": "string",
        "enum": ["Celsius", "Fahrenheit"]
      }
    },
    "required": ["sensorId", "timestamp", "temperature", "unit"]
  }



"""

chat = [
    { "role": "user", "content": prompt }
]


prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1500)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Write the Python code to convert the following JSON FILE using the following JSON SCHEMA:

JSON FILE:

{
    "channel": {
        "id": 1293177,
        "name": "San Diego - Estación Meteorológica",
        "description": "San Diego, Cerro Largo, Uruguay
Estación Meteorológica Solar
(Temp, Hum, Presion, Lluvia, Viento).
ESP8266, UNO R3, BME 680
Update Interval - 15 seg
https://clima.santiago.ovh/",
        "latitude": "-31.9939484",
        "longitude": "-53.9575388",
        "field1": "Temperatura C°",
        "field2": "Humedad %",
        "field3": "Pres. Atmosférica (hPa)",
        "field4": "Gas",
        "field5": "Viento",
        "field6": "Precipitación (mm)",
        "field7": "Dirección del Viento",
        "field8": "UV",
        "created_at": "2021-01-30T16:32:32Z",
        "updated_at": "2024-06-18T14:05:34Z",
        "elevation": "136",
        "last_entry_id": 4502987
    },
    "feeds": [
        {
            "created_at": "2024-06-18T13:33:39

# Let's run the code.

In [6]:
import json

# Read the JSON file
with open("data.json") as f:
    data = json.load(f)

# Convert the JSON data to the specified schema
schema = {
    "type": "object",
    "properties": {
        "sensorId": {"type": "string"},
        "timestamp": {"type": "string", "format": "date-time"},
        "temperature": {"type": "number"},
        "unit": {"type": "string", "enum": ["Celsius", "Fahrenheit"]}
    },
    "required": ["sensorId", "timestamp", "temperature", "unit"]
}

converted_data = json.dumps(data, schema)

# Print the converted data
print(converted_data)

TypeError: dumps() takes 1 positional argument but 2 were given